In [1]:
import os
import re
from time import sleep, time

import bs4
import grequests
import requests
import json
from uptech.utils import chunks
import os
import re

In [2]:
letters = [
    "A",
    "B",
    "V",
    "G",
    "D",
    "E",
    "ZH",
    "Z",
    "I",
    "Y",
    "K",
    "L",
    "M",
    "N",
    "O",
    "P",
    "R",
    "S",
    "T",
    "U",
    "F",
    "H",
    "C",
    "CH",
    "SH",
    "AE",
    "YU",
    "YA",
]
base_url = 'https://meds.is'

In [5]:
resp = requests.get('https://meds.is/alphabet/A')


In [10]:
soup = bs4.BeautifulSoup(resp.content, 'html.parser')

In [15]:
items = soup.find_all(href=lambda h: h and h.startswith('/drugs/'))

In [21]:
items[0].text.strip(), items[0]['href']

('Абаджио', '/drugs/24188-abadzhio')

In [25]:
def parse_letter_page(html):
    soup = bs4.BeautifulSoup(resp.content, 'html.parser')
    items = soup.find_all(href=lambda h: h and h.startswith('/drugs/'))
    return [{'name': it.text.strip(), 'url': base_url + it['href']} for it in items]

In [32]:
data = []

In [33]:
for l in letters:
    resp = requests.get(f'https://meds.is/alphabet/{l}')
    data.extend(parse_letter_page(resp.content))
    print(l)
    sleep(3)

A
B
V
G
D
E
ZH
Z
I
Y
K
L
M
N
O
P
R
S
T
U
F
H
C
CH
SH
AE
YU
YA


In [28]:
print('kek')
print('kek')


kek
kek


In [36]:
with open('drugs.json', 'w') as f:
    f.write(json.dumps(data, indent=2))

In [10]:
with open(f'medsis_dump/{data[0]["url"].split("/")[-1]}.html', 'wb') as f:
    f.write(requests.get(data[0]['url']).content)

In [42]:
len(resp.content)

359583

In [2]:
def exception_handler(request, excepetion):
    raise print(type(excepetion))


rs = [grequests.get('https://meds.is/drugs/25530-ashlev', timeout=10)]
resps = grequests.map(rs, exception_handler=exception_handler)

In [8]:
failed_urls = []
success_urls = []
with open('drugs.json', 'r') as f:
    drugs = json.load(f)


def exception_handler(request, exception):
    failed_urls.append(request.url)

In [9]:
cnt = 0
for drug_chunk in chunks(drugs, 10):
    rs = [grequests.get(d['url'], timeout=10) for d in drug_chunk]
    resps = grequests.map(rs, exception_handler=exception_handler)
    for resp in resps:
        if resp and resp.status_code == 200:
            with open(f'medsis_dump/{resp.url.split("/")[-1]}.html', 'wb') as f:
                f.write(resp.content)
            success_urls.append(resp.url)

    cnt += len(drug_chunk)
    if cnt % 100 == 0:
        print(f'Successes: {len(success_urls)}, fails: {len(failed_urls)}')
        sleep(3)

Successes: 100, fails: 0
Successes: 200, fails: 0
Successes: 300, fails: 0
Successes: 400, fails: 0
Successes: 500, fails: 0
Successes: 600, fails: 0
Successes: 700, fails: 0
Successes: 800, fails: 0
Successes: 900, fails: 0
Successes: 1000, fails: 0
Successes: 1100, fails: 0
Successes: 1200, fails: 0
Successes: 1300, fails: 0
Successes: 1400, fails: 0
Successes: 1500, fails: 0
Successes: 1600, fails: 0
Successes: 1700, fails: 0
Successes: 1800, fails: 0
Successes: 1900, fails: 0
Successes: 2000, fails: 0
Successes: 2100, fails: 0
Successes: 2200, fails: 0
Successes: 2300, fails: 0
Successes: 2400, fails: 0
Successes: 2500, fails: 0
Successes: 2600, fails: 0
Successes: 2700, fails: 0
Successes: 2800, fails: 0
Successes: 2900, fails: 0
Successes: 3000, fails: 0
Successes: 3100, fails: 0
Successes: 3200, fails: 0
Successes: 3300, fails: 0
Successes: 3400, fails: 0
Successes: 3500, fails: 0
Successes: 3600, fails: 0
Successes: 3700, fails: 0
Successes: 3800, fails: 0
Successes: 3900, fail

In [46]:
drug_url_to_name = {d['url']: d['name'] for d in drugs}

In [47]:
def parse_drug(file_name):
    with open(f'medsis_dump/{file_name}', 'rb') as f:
        soup = bs4.BeautifulSoup(f.read(), 'html.parser')

    result = {
        'medsis_id': int(file_name.split('-')[0]),
        'medsis_url': f'https://meds.is/drugs/{file_name.split(".")[0]}'
    }
    result['name'] = drug_url_to_name[result['medsis_url']]
    stat_fields = [
        'effectiveness',
        'safety',
        'convenience',
        'contraindications',
        'side_effects',
        'tolerance',
    ]
    progress_values = soup.find_all('progress')
    assert len(progress_values) == 6, f'Invalid parsed progress values: {file_name}'

    for stat, tag in zip(stat_fields, progress_values):
        try:
            result[stat] = int(tag['value'])
        except:
            assert False, f'Cannot parse stat {stat} at tag {tag}'

    score_div = soup.find('div', {'class': 'level-item digits'})
    assert score_div is not None, f'Score div not found in {file_name}'
    score_parts = score_div.findChildren('div', {'class': 'num'})
    assert len(score_parts) == 2, f'Score parts are invalid at div {score_div} at file {file_name}'
    try:
        float(score_parts[0].text)
    except:
        assert False, f'Invalid score: {score_parts[0].text} at div {score_div} at file {file_name}'
    result['score'] = score_parts[0].text

    analogues_hrefs = [item['href'] for item in soup.findAll('a', {'style': 'position:relative;display:none;'})]
    analogue_medsis_ids = []
    for href in analogues_hrefs:
        try:
            medsis_id = int(href.split('/')[-1].split('-')[0])
        except:
            assert False, f'Bad analogue href: {href} at file {file_name}'
        analogue_medsis_ids.append(medsis_id)

    result['analogue_medsis_ids'] = analogue_medsis_ids
    return result

In [49]:
parse_drug('1-siofor.html')

{'medsis_id': 1,
 'medsis_url': 'https://meds.is/drugs/1-siofor',
 'name': 'Сиофор 500',
 'effectiveness': 82,
 'safety': 79,
 'convenience': 81,
 'contraindications': 24,
 'side_effects': 23,
 'tolerance': 0,
 'score': '7.3',
 'analogue_medsis_ids': [31452,
  31460,
  31461,
  31453,
  27469,
  34921,
  31457,
  31329,
  31455,
  31454,
  32356,
  31456,
  31458,
  31459]}

In [53]:
parsed_drug_data = []
for cnt, file_name in enumerate(os.listdir('./medsis_dump')):
    parsed_drug_data.append(parse_drug(file_name))
    if cnt % 200 == 0:
        print(cnt)

0
200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3800
4000
4200
4400
4600
4800
5000
5200
5400
5600
5800
6000
6200
6400
6600
6800
7000
7200
7400
7600
7800
8000
8200
8400
8600
8800
9000
9200
9400
9600
9800
10000
10200
10400
10600
10800
11000
11200
11400
11600
11800
12000
12200
12400
12600
12800
13000
13200
13400
13600
13800
14000
14200
14400


In [55]:
with open('parsed_drub_data.json', 'w') as f:
    json.dump(parsed_drug_data, f, indent=2)

In [18]:
with open('HackData/products.json', 'r') as f:
    products = json.load(f)
with open('parsed_drub_data.json', 'r') as f:
    parsed_drug_data = json.load(f)

In [19]:
pattern = re.compile(r', ?| ?- ?|\. ?| +')
sber_cache = {}
medsis_cache = {}
def check_name(sber, medsis):
    if sber not in sber_cache:
        v = [s.strip() for s in re.sub(pattern, ' ', sber.lower()).split() if s.strip()]
        sber_cache[sber] = v
        sber = v
    else:
        sber = sber_cache[sber]
    if medsis not in medsis_cache:
        v = [s.strip() for s in re.sub(pattern, ' ', medsis.lower()).split() if s.strip()]
        medsis_cache[medsis] = v
        medsis = v
    else:
        medsis = medsis_cache[medsis]

    for cnt, (s, m) in enumerate(zip(sber, medsis), start=1):
        if s == m:
            continue
        return cnt - 1, False
    else:
        return min(len(sber), len(medsis)), True

    return 0, False

In [20]:
for cnt, parsed_drug in enumerate(parsed_drug_data):
    for p in products:
        score, full = check_name(p['NAME'], parsed_drug['name'])
        if score:
            if not p.get('medsis') or p['medsis']['score'] < score or p['medsis']['score'] == score and full:
                p['medsis'] = {'ids': [parsed_drug['medsis_id']], 'score': score, 'full': full}
            elif p.get('medsis') and p['medsis']['score'] == score and not p['medsis']['full']:
                p['medsis']['ids'].append(parsed_drug['medsis_id'])

    if cnt % 100 == 0:
        print(cnt)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500


In [21]:
print(len(products))
print(sum(1 for p in products if not p.get('medsis')))
print(sum(1 for p in products if len(p.get('medsis', {}).get('ids', [])) == 1))
print(sum(1 for p in products if len(p.get('medsis', {}).get('ids', [])) > 1))

40022
29788
9097
1137


In [27]:
medsis_id_map = [{'sber_id': p['ID'], 'medsis_ids': p.get('medsis', {}).get('ids', [])} for p in products]
with open('medsis_id_map.json', 'w') as f:
    json.dump(medsis_id_map, f)

In [60]:
print(len(products))
print(sum(1 for p in products if p['medsis_ids'] == 0))
print(sum(1 for p in products if p['medsis_ids'] == 1))
print(sum(1 for p in products if len(p['medsis_ids']) > 1))
[p for p in products if len(p['medsis_ids']) > 1]

40022
0
0
3205


[{'ID': 461374,
  'NAME': 'Далацин, крем вагинальный 2% , 20 г+3 аппликатора',
  'medsis_ids': [1119, 25390]},
 {'ID': 461375,
  'NAME': 'Далацин, крем вагинальный 2% , 40 г+7 аппликаторов',
  'medsis_ids': [1119, 25390]},
 {'ID': 461395,
  'NAME': 'Тамоксифен, таблетки 10 мг, 30 шт.',
  'medsis_ids': [2700, 31666]},
 {'ID': 461396,
  'NAME': 'Тамоксифен, таблетки 20 мг, 30 шт.',
  'medsis_ids': [2700, 31666]},
 {'ID': 461397,
  'NAME': 'Тамоксифен, таблетки 20 мг, 100 шт.',
  'medsis_ids': [2700, 31666]},
 {'ID': 461399,
  'NAME': 'Офтан Дексаметазон капли глазные 1 мг/мл, 5 мл',
  'medsis_ids': [1123, 32911]},
 {'ID': 461402,
  'NAME': 'Фотил форте капли глазные 40 мг/мл + 5 мг/мл, 5 мл',
  'medsis_ids': [1651, 1650]},
 {'ID': 461404,
  'NAME': 'Офтан Тимолол капли глазные 5 мг/мл, 5 мл',
  'medsis_ids': [1916, 254, 35824]},
 {'ID': 461415,
  'NAME': 'Дульколакс таблетки покрыт.об. 5 мг, 30 шт.',
  'medsis_ids': [551, 1176]},
 {'ID': 461429,
  'NAME': 'Флюдитек, сироп 5% , 125 мл',
 

In [61]:
p1 = products

In [93]:
check_name('Виагра, таблетки 100 мг, 2 шт.', 'Пароксетин-СЗ')

2

2

In [96]:
medsis_cache

{'Карнифит': ['карнифит'],
 'Беклоспир': ['беклоспир'],
 'Топирамат-АЛСИ': ['топирамат', 'алси'],
 'Рекорд В12 Сигма-Тау': ['рекорд', 'в12', 'сигма', 'тау'],
 'Кандекор Н32': ['кандекор', 'н32'],
 'Стронция-89 хлорид': ['стронция', '89', 'хлорид'],
 'Железо': ['железо'],
 'Фитогепатол №2': ['фитогепатол', '№2'],
 'Ридостин': ['ридостин'],
 'Флюоресцит': ['флюоресцит'],
 'Зильден': ['зильден'],
 'Полижинакс Вирго': ['полижинакс', 'вирго'],
 'Магний плюс В6': ['магний', 'плюс', 'в6'],
 'Никетамид-Эском': ['никетамид', 'эском'],
 'Камфорный спирт и муравьиный спирт поровну': ['камфорный',
  'спирт',
  'и',
  'муравьиный',
  'спирт',
  'поровну'],
 'Гидрокортизон + Натамицин + Неомицин': ['гидрокортизон',
  '+',
  'натамицин',
  '+',
  'неомицин'],
 'Атропин': ['атропин'],
 'Зенаро': ['зенаро'],
 'Синдроксоцин': ['синдроксоцин'],
 'Этамбутол ШТАДА': ['этамбутол', 'штада'],
 'Гигротон': ['гигротон'],
 'Данол': ['данол'],
 'Бусерелин': ['бусерелин'],
 'Пектрол': ['пектрол'],
 'Нитрофунгин': 